#### AutoEncoder란?
- 비지도 방식을 사용해서 데이터코딩을 학습하는데 사용되는 신경망
- 입력을 출력으로 복사하는 신경망, 네트워크의 layer를 쌓으면서 데이터를 압축하거나 입력 데이터의 노이즈를 제거해서 깔끔한 이미지로 복원하는 등의 다양한 오토인코더 존재


- 오토인코더는 데이터의 노이즈를 제거하거나, 데이터의 시각화를 위한 차원 축소에 적절하게 사용될 수는 있다.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets

In [ ]:
if torch.cuda.is_available():
  DEVICE=torch.device('cuda')
else:
  DEVICE=torch.device('cpu')
print('Using PyTorch version:',torch.__version__,'Device:',DEVICE)

In [ ]:
BATCH_SIZE=32
EPOCHS=10

In [ ]:
train_dataset=datasets.FashionMNIST(root='../data/MNIST',
                                    train=True,
                                    download=True,
                                    transform=transforms.ToTensor())
test_dataset=datasets.FashionMNIST(root='../data/MNIST',
                                   train=True,
                                   transform=transforms.ToTensor())
train_loader=torch.utils.data.DataLoader(dataset=train_dataset,
                                         batch_size=BATCH_SIZE,
                                         shuffle=True)
test_loader=torch.utils.data.DataLoader(dataset=test_dataset,
                                        batch_size=BATCH_SIZE,
                                        shuffle=False)

In [ ]:
for (X_train,y_train) in train_loader:
  print('X_train:',X_train.size(),'type:',X_train.type())
  print('y_train:',y_train.size(),'type:',y_train.type())
  break

In [ ]:
pltsize=1
plt.figure(figsize=(10*pltsize,pltsize))
for i in range(10):
  plt.subplot(1,10,i+1)
  plt.axis('off')
  plt.imshow(X_train[i,:,:,:].numpy().reshape(28,28),cmap='gray_r')
  plt.title('Class:'+str(y_train[i].item()))

In [ ]:
#AutoEncoder(AE) 모델 설계
class AE(nn.Module):
  def __init__(self):
    super(AE,self).__init__()

    self.encoder=nn.Sequential(
        nn.Linear(28*28,512),
        nn.ReLU(),
        nn.Linear(512,256),
        nn.ReLU(),
        nn.Linear(256,32),
    )

    self.decoder=nn.Sequential(
        nn.Linear(32,256),
        nn.ReLU(),
        nn.Linear(256,512),
        nn.ReLU(),
        nn.Linear(512,28*28),
    )

  def forward(self,x):
    encoded=self.encoder(x)
    decoded=self.decoder(encoded)
    return encoded,decoded

In [ ]:
model=AE().to(DEVICE)
optimizer=torch.optim.Adam(model.parameters(),lr=0.01)
criterion=nn.MSELoss()

print(model)

In [ ]:
def train(model, train_loader, optimizer, log_interval):
    model.train()
    for batch_idx, (image, _) in enumerate(train_loader):
        image = image.view(-1, 28 * 28).to(DEVICE)
        target = image.view(-1, 28 * 28).to(DEVICE)
        optimizer.zero_grad()
        encoded, decoded = model(image)
        loss = criterion(decoded, target)
        loss.backward()
        optimizer.step()

        if batch_idx % log_interval == 0:
            print("Train Epoch: {} [{}/{} ({:.0f}%)]\tTrain Loss: {:.6f}".format(
                epoch, batch_idx * len(image), 
                len(train_loader.dataset), 100. * batch_idx / len(train_loader), 
                loss.item()))

In [ ]:
def evaluate(model,test_loader):
    model.eval()
    test_loss=0
    real_image=[]
    gen_image=[]
    with torch.no_grad():
      for image,_ in test_loader:
        image=image.view(-1,28*28).to(DEVICE)
        target=image.view(-1,28*28).to(DEVICE)
        encoded,decoded=model(image)

        test_loss+=criterion(decoded,image).item()
        real_image.append(image.to('cpu'))
        gen_image.append(decoded.to('cpu'))
    test_loss /= (len(test_loader.dataset) / BATCH_SIZE)

    return test_loss, real_image, gen_image

In [ ]:
# AutoEncoder 학습 실행하며 Test set의 Reconstruction Error 확인하기
for epoch in range(1,EPOCHS+1):
    train(model,train_loader,optimizer,log_interval=200)
    test_loss,real_image,gen_image=evaluate(model,test_loader)
    print("\n[EPOCH: {}], \tTest Loss: {:.4f}".format(epoch, test_loss))
    f, a = plt.subplots(2, 10, figsize = (10, 4))
    for i in range(10):
        img = np.reshape(real_image[0][i], (28, 28))
        a[0][i].imshow(img, cmap = "gray_r")
        a[0][i].set_xticks(())
        a[0][i].set_yticks(())
    
    for i in range(10):
        img = np.reshape(gen_image[0][i], (28, 28))
        a[1][i].imshow(img, cmap = "gray_r")
        a[1][i].set_xticks(())
        a[1][i].set_yticks(())
    plt.show()